<center>
<img src='https://camo.githubusercontent.com/ad20aaaa48264581c46aa126919dbb72060ebd98cffe015523549b4974843ae5/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f46556c79616e6b696e2f725f70726f626162696c6974792f6d61737465722f656e645f73656d696e6172732f73656d5f362f73747564656e745f746561636865725f646973742e706e67' align='center' width="800x">
</center>

# Андан на экономе

## Семинар 11:  гипотетический семинар

In [2]:
import numpy as np
import pandas as pd

from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

### Упражнение 1 (AB-тест)

- задачка на проверку гипотез о долях/средних как будто AB-тест

### Упражнение 2 (ошибки 1 и 2 рода)

### Упражнение 3 (распределение p-value)

- задачка на распределение pvalue при разных уровнях значимости


Вовочка тестирует гипотезу 
 против гипотезы 
. Предположим, что 
 на самом деле верна. По своей сути  является случайной величиной. Какое распределение оно имеет?

### Упражнение 4 (уровень значимости выходит из-под контроля)

- задачка про выход уровня значимости из-под контроля


### Упражнение 5 (пересечение Vs разность)

- задачка про мощность критериев с д.и. и без них по-нормальному

In [ ]:
- придумать задачу о том, что нормальности средних недостаточно!